# IRAF-like source detection with simulated images

In [ ]:
from astropy.stats import sigma_clipped_stats, gaussian_sigma_to_fwhm
from astropy.table import QTable
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.datasets import make_100gaussians_image, make_gaussian_sources_image
from photutils.detection import find_peaks


In [ ]:
data = make_100gaussians_image()
mean, median, std = sigma_clipped_stats(data, sigma=3.0)

In [ ]:
median, std

In [ ]:
from photutils.detection import DAOStarFinder
daofind = DAOStarFinder(fwhm=10.0, threshold=5.*std, ratio=.33, theta=0)  
sources = daofind(data - median)  
for col in sources.colnames:  
    if col not in ('id', 'npix'):
        sources[col].info.format = '%.2f'  # for consistent table output
sources.pprint(max_width=76)  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.0)
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1.5, alpha=0.5);

## Sources in image

In [ ]:
n_sources = 100
flux_range = [500, 1000]
xmean_range = [0, 500]
ymean_range = [0, 300]
xstddev_range = [1, 5]
ystddev_range = [1, 5]
params = {'flux': flux_range,
          'x_mean': xmean_range,
          'y_mean': ymean_range,
          'x_stddev': xstddev_range,
          'y_stddev': ystddev_range,
          'theta': [0, 2 * np.pi]}

rng = np.random.RandomState(12345)
inp_sources = QTable()
for param_name, (lower, upper) in params.items():
    # Generate a column for every item in param_ranges, even if it
    # is not in the model (e.g., flux).  However, such columns will
    # be ignored when rendering the image.
    inp_sources[param_name] = rng.uniform(lower, upper, n_sources)
xstd = inp_sources['x_stddev']
ystd = inp_sources['y_stddev']
inp_sources['amplitude'] = inp_sources['flux'] / (2.0 * np.pi * xstd * ystd)

In [ ]:
inp_sources.pprint(max_width=76)  


In [ ]:
plt.figure(figsize=(20, 10))
inp_positions = np.transpose((inp_sources['x_mean'], inp_sources['y_mean']))
inp_apertures = CircularAperture(inp_positions, r=10.0)
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1.5, alpha=0.5);
inp_apertures.plot(color='green', lw=1, alpha=0.5);
for source in inp_sources:
    minor = min(source['x_stddev'], source['y_stddev'])
    major = max(source['x_stddev'], source['y_stddev'])
    plt.annotate(
        f"{minor / major:.2f}", 
        #f"{1 - source['x_stddev'] / source['y_stddev']:.2f}", 
        (source['x_mean'] + 5, source['y_mean'] + 5)
    )
    plt.annotate(
        f"{(source['x_stddev'] + source['x_stddev'])/2 * gaussian_sigma_to_fwhm:.2f}", 
        (source['x_mean'] + 5, source['y_mean'] - 5)
    )
    # plt.annotate(
    #     f"{source['amplitude']:.2f}", 
    #     (source['x_mean'] + 5, source['y_mean'] - 5)
    # )


In [ ]:
daofind_source = DAOStarFinder(fwhm=5.0, threshold=5.*std, xycoords=inp_positions)  
sources_source = daofind_source(data - median)

In [ ]:
sources_source

## `find_peaks`

In [ ]:
threshold = median + (5.0 * std)
tbl = find_peaks(data, threshold, box_size=11)
tbl['peak_value'].info.format = '%.8g'  # for consistent table output
tbl.pprint(max_width=76)  

In [ ]:
positions = np.transpose((tbl['x_peak'], tbl['y_peak']))
apertures = CircularAperture(positions, r=5.0)
#norm = simple_norm(data, 'sqrt', percent=99.9)
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='#0547f9', lw=1.5)
plt.xlim(0, data.shape[1] - 1)
plt.ylim(0, data.shape[0] - 1)

## Sources all theta=0

In [ ]:
flat_inp_sources = inp_sources.copy()
flat_inp_sources['theta'] = 0.0
minor_x = flat_inp_sources['x_stddev'] < flat_inp_sources['y_stddev']
majors = flat_inp_sources['x_stddev'].copy()
majors[minor_x] = flat_inp_sources['y_stddev'][minor_x]

minors = flat_inp_sources['y_stddev'].copy()
minors[minor_x] = flat_inp_sources['x_stddev'][minor_x]

flat_inp_sources['x_stddev'] = majors
flat_inp_sources['y_stddev'] = minors

flat_inp_sources['y_stddev'] = flat_inp_sources['x_stddev']

In [ ]:
shape = (300, 500)
flat_data = make_gaussian_sources_image(shape, flat_inp_sources) + 5.0

if True:
    rng = np.random.RandomState(12345)
    flat_data += rng.normal(loc=0.0, scale=2.0, size=shape)



In [ ]:
plt.figure(figsize=(20, 10))

plt.imshow(flat_data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')

In [ ]:
daofind = DAOStarFinder(fwhm=10.0, threshold=5.*std, ratio=1, sharplo=0.2)  
flat_sources = daofind(flat_data - median) 

In [ ]:
plt.figure(figsize=(20, 10))

flat_positions = np.transpose((flat_sources['xcentroid'], flat_sources['ycentroid']))
flat_apertures = CircularAperture(flat_positions, r=4.0)

plt.imshow(flat_data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
flat_apertures.plot(color='blue', lw=1.5, alpha=0.5);
for source in flat_inp_sources:
    minor = min(source['x_stddev'], source['y_stddev'])
    major = max(source['x_stddev'], source['y_stddev'])
    plt.annotate(
        f"{minor / major:.2f}", 
        #f"{1 - source['x_stddev'] / source['y_stddev']:.2f}", 
        (source['x_mean'] + 5, source['y_mean'] + 5)
    )
    plt.annotate(
        f"{source['amplitude']:.2f}", 
        (source['x_mean'] + 5, source['y_mean'] - 5)
    )
    plt.annotate(
        f"{(source['x_stddev'] + source['x_stddev'])/2 * gaussian_sigma_to_fwhm:.2f}", 
        (source['x_mean'] + 5, source['y_mean'])
    )

In [ ]:
flat_sources.colnames